# Causal Analysis of Carnatic Music: Surrogate Analysis

This notebook features the updated code for Causal inference of Carnatic music compositions.
We rely on surrogate compositions generated using single order Markov Chains to
verify our results/solidify our theory. `utils.generators.markovian` is a module that
decodes a _raga_ as a Markov chain and uses it's stationary distribution to generate
surrogate compositions. The flow of logic is as follows:

- <u>*Step 1: Data Extraction*</u>: Parsed notations are extracted from the respective file in `dataset.BASE_PATH` (`dataset` is one of the import scripts). The files under `dataset.BASE_PATH` that are relevant are saved as `<ragaId>/<composition_name>-parsed.txt`
- <u>*Step 2: Surrogate Generation*</u>: The _raga_ is visualised as a Markov Chain, and using its stationary state, new compositions are generated. These new compositions are then added to the dataset.
- <u>*Step 3: Data Processing*</u>: The whole dataset then undergoes conversion to adjusted melody, i.e. the pitch indices are repeated a particular number of times proportional to the note-event duration,
- <u>*Step 4: Causal Analysis*</u>: The adjusted melodies are then chosen pairwise, and every pair is used as an argument for `ETC.CCM_Causality()`. In this manner, causality is inferred for each pair.

In [1]:
import time

from utils import dataStructures, dataset, parallelizer
from utils.generators import markovian
import numpy as np
import os

A lambda function to calculate the average of a given array/list

In [2]:
avg_conns = lambda t_c : np.array(t_c).mean()

In [ ]:
# utility function; should only be invoked inside a function
def _print_correct(mela_id='22', janya_id='22_a', num_surr=100, surr_serial=False, time_cal=None, pre_length=None):
    # Data Extraction
    dat = dataset.GetRagaSongCoordsConcat(mela_id, janya_id)
    # dat1 = dataset.GetRagaSongCoords(janya_id)
    if time_cal is not None:
        assert isinstance(time_cal, dict)
    print("Starting surrogate generation...")
    t1 = time.perf_counter()

    # Surrogate Generation
    d_surr1 = markovian.GenerateForRaga(mela_id, num_surr, serial=surr_serial)
    d_surr2 = markovian.GenerateForRaga(janya_id, num_surr, serial=surr_serial)
    t2 = time.perf_counter()
    for key in dat:
        dat[key] = dataStructures.PackTuples(*dat[key])

    # for key in dat1:
    #     dat1[key] = dataStructures.PackTuples(*dat1[key])

    d_fin1 = dataset.ConcatenateDicts(dat, d_surr1, d_surr2)

    # Data Processing
    adj = dataset.GetAdjustedMelodies(d_fin1, pre_min_len=pre_length)
    print(f"adjusted Melody length: {len(list(adj.values())[0])}")
    print("finding causality....")
    t3 = time.perf_counter()

    # Causal Analysis
    tc, totc, strengths = parallelizer.TrueLZPCausality_listParallel(list(adj.values()), list(adj.keys()), mela=dataset.GetRagaFromRagaId(mela_id))
    t4 = time.perf_counter()

    if time_cal is not None:
        time_cal["Surrogate{}".format(mela_id)] = t2 - t1
        time_cal["Causality{}".format(mela_id)] = t4 - t3
    return tc, totc, strengths

#Deprecated
# def get_generation_time50(mela_id='22', janya_id='22_a', surr_serial=True):
#     dat = dataset.GetRagaSongCoordsConcat(mela_id)
#     dat1 = dataset.GetRagaSongCoords(janya_id)
#     print("Starting surrogate generation...")
#     t1 = time.perf_counter()
#     d_surr1 = markovian.GenerateForRaga(mela_id, 50, serial=surr_serial)
#     d_surr2 = markovian.GenerateForRaga(janya_id, 50, serial=surr_serial)
#     t2 = time.perf_counter()
#     for key in dat:
#         dat[key] = dataStructures.PackTuples(*dat[key])
#
#     for key in dat1:
#         dat1[key] = dataStructures.PackTuples(*dat1[key])
#
#     d_fin1 = dataset.ConcatenateDicts(dat, d_surr1, dat1, d_surr2)
#     adj = dataset.GetAdjustedMelodies(d_fin1)
#
#     return t2 - t1

# utility function; should only be invoked inside a function
def _ragaIterator(n_iters, mela, janya, num_surrogates=100, write=True, times_it=None, pre_length=None):
    if times_it is not None:
        assert isinstance(times_it, dict)
        times_it["Surrogate{}".format(mela)] = []
        times_it["Causality{}".format(mela)] = []
    p = pre_length if pre_length is not None else ''
    tcs = []
    strengths = []
    totcs = []
    f = open("/home/efm-workstation/surr_stats/{}/truecauses{}_{}.txt".format(mela, num_surrogates, p), "w")
    t1 = time.perf_counter()
    for i in range(n_iters):
        print("iteration: ", i + 1)
        tm = {}
        tc, totc, st = _print_correct(mela, janya, num_surr=num_surrogates, surr_serial=True, time_cal=tm, pre_length=pre_length)

        if times_it is not None:
            times_it["Surrogate{}".format(mela)].append(tm["Surrogate{}".format(mela)])
            times_it["Causality{}".format(mela)].append(tm["Causality{}".format(mela)])
        if write:
            f.write("iteration: {}/{}, {}/{}; time taken: {}/{}\n".format(i+1, n_iters, tc, totc,
                                                                          tm["Surrogate{}".format(mela)],
                                                                          tm["Causality{}".format(mela)]))
        t = time.perf_counter()
        print("time elapsed: ", (t - t1))
        t1 = t
        tcs.append(tc)
        totcs.append(totc)
        strengths.append(st)
    f.write("\n Average Connections: {}/{};\t Avg. time taken: {}/{}\n".format(avg_conns(tcs), totcs[-1],
                                                                               avg_conns(times_it["Surrogate{}".format(mela)]),
                                                                               avg_conns(times_it["Causality{}".format(mela)])))
    f.close()
    return tcs, totcs, strengths

The `run` function defined below is used to "run" the experiments of Causal Analysis.
The arguments to this function are the required *Melakarta* and *Janya raga*, number
of surrogate compositions to be generated per raga (0 means no surrogates will be generated),
the number of iteration the experiment needs to be repeated, and a pre-defined constant length,
which is usually helpful if it is desirable to have all the compositions across all the
*raga* to be of the sane length.

> Note: the length of a composition, as explained in the article as well, is the total number of note events in it.

In [ ]:
def run(mela, janya, num_surrogates=50, num_iterations=10, const_len=None):
    t_dict = {}
    tc, totc, ss = _ragaIterator(num_iterations, mela, janya, num_surrogates=num_surrogates, times_it=t_dict, pre_length=const_len)
    os.system("cat /home/efm-workstation/surr_stats/{}/truecauses{}.txt".format(mela, janya))
    print(f"{avg_conns(tc)}/{totc[-1]}, {avg_conns(t_dict['Surrogate{}'.format(mela)])}, {avg_conns(t_dict['Causality{}'.format(mela)])}")

## 100 Surrogates

In [4]:
run('8', '8_d', num_surrogates=100, const_len=43680)
run('15', '15_m', num_surrogates=100, const_len=43680)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [00:40<00:00,  2.46it/s]


adjusted Melody length: 43680
finding causality....
1523 true causes out of 11448 total connections
time elapsed:  811.9294139250123
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:38<00:00,  2.62it/s]


adjusted Melody length: 43680
finding causality....
1432 true causes out of 11448 total connections
time elapsed:  800.6072855200036
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
8791 true causes out of 11024 total connections
time elapsed:  657.3160438409832
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [02:13<00:00,  1.33s/it]


adjusted Melody length: 43680
finding causality....
8320 true causes out of 11024 total connections
time elapsed:  735.6349998739897
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [00:41<00:00,  2.39it/s]


adjusted Melody length: 43680
finding causality....
8380 true causes out of 11024 total connections
time elapsed:  657.1555884560221
8232.8/11024, 127.85673798049683, 557.1754275367014


In [5]:
run('22', '22_a', num_surrogates=100, const_len=43680)
run('28', '28_k', num_surrogates=100, const_len=43680)
run('29', '29_h', num_surrogates=100, const_len=43680)
run('65', '29_h', num_surrogates=100, const_len=43680)

iteration:  1
Starting surrogate generation...


100%|██████████| 100/100 [02:41<00:00,  1.61s/it]


adjusted Melody length: 43680
finding causality....
7371 true causes out of 10918 total connections
time elapsed:  1009.0824446390034
iteration:  2
Starting surrogate generation...


100%|██████████| 100/100 [00:29<00:00,  3.38it/s]


adjusted Melody length: 43680
finding causality....
6635 true causes out of 10918 total connections
time elapsed:  697.2316013179952
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
5726 true causes out of 11024 total connections
time elapsed:  826.9733312440221
iteration:  9
Starting surrogate generation...


100%|██████████| 100/100 [00:36<00:00,  2.76it/s]


adjusted Melody length: 43680
finding causality....
6058 true causes out of 11024 total connections
time elapsed:  695.3973450609774
iteration:  10
Starting surrogate generation...


100%|██████████| 100/100 [02:59<00:00,  1.80s/it]


adjusted Melody length: 43680
finding causality....
6365 true causes out of 11024 total connections
time elapsed:  838.625210269005
5957.2/11024, 115.58340713129728, 589.8887463447987


## 50 Surrogates

In [3]:
run('8', '8_d', num_surrogates=50, const_len=43680)
run('15', '15_m', num_surrogates=50, const_len=43680)
run('22', '22_a', num_surrogates=50, const_len=43680)
run('28', '28_k', num_surrogates=50, const_len=43680)
run('29', '29_h', num_surrogates=50, const_len=43680)
run('65', '29_h', num_surrogates=50, const_len=43680)


iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:15<00:00,  3.23it/s]


adjusted Melody length: 43680
finding causality....
488 true causes out of 3248 total connections
time elapsed:  283.5978773999959
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:15<00:00,  3.21it/s]


adjusted Melody length: 43680
finding causality....
539 true causes out of 3248 total connections
time elapsed:  251.5753110700025
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
1585 true causes out of 3024 total connections
time elapsed:  219.38133629200456
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:14<00:00,  3.34it/s]


adjusted Melody length: 43680
finding causality....
1691 true causes out of 3024 total connections
time elapsed:  209.76835732099426
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [03:48<00:00,  4.58s/it]


adjusted Melody length: 43680
finding causality....
1801 true causes out of 3024 total connections
time elapsed:  423.2699810220074
1716.9/3024, 56.66018024840014, 160.79365222810011


# Warning! All that follows is outdated

In [4]:
t_dict = {}

In [11]:
tc65, totc65, ss65 = _ragaIterator(10, '65', '29_h', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:12<00:00,  3.96it/s]


adjusted Melody length: 43680
finding causality....
1615 true causes out of 3024 total connections
time elapsed:  218.37166827799956
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:21<00:00,  2.38it/s]


adjusted Melody length: 43680
finding causality....
1669 true causes out of 3024 total connections
time elapsed:  240.33316805800132
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
1747 true causes out of 3024 total connections
time elapsed:  212.7220547470024
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:20<00:00,  2.41it/s]


adjusted Melody length: 43680
finding causality....
1608 true causes out of 3024 total connections
time elapsed:  233.68813059999957
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:14<00:00,  3.53it/s]


adjusted Melody length: 43680
finding causality....
1480 true causes out of 3024 total connections
time elapsed:  219.93920594000156


In [12]:
avg_conns(tc65), avg_conns(t_dict["Surrogate65"]), avg_conns(t_dict["Causality65"])

(1644.6, 37.424173020300074, 182.3727227753996)

In [5]:
tc15, totc15, ss15 = _ragaIterator(10, '15', '15_m', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:13<00:00,  3.79it/s]


adjusted Melody length: 43680
finding causality....
2633 true causes out of 3024 total connections
time elapsed:  212.19028972700016
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:21<00:00,  2.34it/s]


adjusted Melody length: 43680
finding causality....
2341 true causes out of 3024 total connections
time elapsed:  236.07045239299987
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
2431 true causes out of 3024 total connections
time elapsed:  208.76942676599992
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:14<00:00,  3.52it/s]


adjusted Melody length: 43680
finding causality....
2307 true causes out of 3024 total connections
time elapsed:  189.96863819200007
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:17<00:00,  2.85it/s]


adjusted Melody length: 43680
finding causality....
2211 true causes out of 3024 total connections
time elapsed:  198.76770169299925


In [10]:
# os.system("cat /home/efm-workstation/surr_stats/{}/truecauses{}.txt".format(15, 50))
avg_conns(tc15), avg_conns(t_dict["Surrogate15"]), avg_conns(t_dict["Causality15"])

(2417.3, 55.514034086399946, 148.08025281639988)

In [15]:
tc8, totc8, ss8 = _ragaIterator(10, '8', '8_d', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:16<00:00,  2.99it/s]


adjusted Melody length: 43680
finding causality....
619 true causes out of 3248 total connections
time elapsed:  270.9603171999988
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:18<00:00,  2.70it/s]


adjusted Melody length: 43680
finding causality....
782 true causes out of 3248 total connections
time elapsed:  278.37013650600056
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
612 true causes out of 3248 total connections
time elapsed:  276.27639988900046
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:17<00:00,  2.89it/s]


adjusted Melody length: 43680
finding causality....
553 true causes out of 3248 total connections
time elapsed:  259.0935400140006
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:30<00:00,  1.63it/s]


adjusted Melody length: 43680
finding causality....
703 true causes out of 3248 total connections
time elapsed:  288.6824951169983


In [16]:
avg_conns(tc8), avg_conns(t_dict["Surrogate8"]), avg_conns(t_dict["Causality8"])

(668.6, 40.217083348899905, 223.51942214590053)

In [17]:
tc22, totc22, ss22 = _ragaIterator(10, '22', '22_a', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:18<00:00,  2.70it/s]


adjusted Melody length: 43680
finding causality....
1883 true causes out of 2968 total connections
time elapsed:  226.83664754599886
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:14<00:00,  3.39it/s]


adjusted Melody length: 43680
finding causality....
2022 true causes out of 2968 total connections
time elapsed:  230.5618512630026
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
2053 true causes out of 2968 total connections
time elapsed:  221.2663969109999
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:12<00:00,  3.89it/s]


adjusted Melody length: 43680
finding causality....
1665 true causes out of 2968 total connections
time elapsed:  225.04745852699853
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:12<00:00,  4.10it/s]


adjusted Melody length: 43680
finding causality....
1835 true causes out of 2968 total connections
time elapsed:  214.0779628510063


In [18]:
avg_conns(tc22), avg_conns(t_dict["Surrogate22"]), avg_conns(t_dict["Causality22"])

(1899.4, 33.20455161340105, 182.73843879880042)

In [19]:
tc28, totc28, ss28 = _ragaIterator(10, '28', '28_k', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:19<00:00,  2.58it/s]


adjusted Melody length: 43680
finding causality....
1583 true causes out of 2970 total connections
time elapsed:  246.19132544999593
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:24<00:00,  2.02it/s]


adjusted Melody length: 43680
finding causality....
1491 true causes out of 2970 total connections
time elapsed:  265.5274024860046
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
1428 true causes out of 2970 total connections
time elapsed:  252.37233264899987
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:18<00:00,  2.77it/s]


adjusted Melody length: 43680
finding causality....
1428 true causes out of 2970 total connections
time elapsed:  248.53994721799972
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:20<00:00,  2.43it/s]


adjusted Melody length: 43680
finding causality....
1588 true causes out of 2970 total connections
time elapsed:  254.24143342200114


In [20]:
avg_conns(tc28), avg_conns(t_dict["Surrogate28"]), avg_conns(t_dict["Causality28"])

(1493.6, 39.67997062440045, 208.65607820050136)

In [21]:
tc29, totc29, ss29 = _ragaIterator(10, '29', '29_h', num_surrogates=50, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 50/50 [00:17<00:00,  2.93it/s]


adjusted Melody length: 43680
finding causality....
2319 true causes out of 3024 total connections
time elapsed:  243.95732997899904
iteration:  2
Starting surrogate generation...


100%|██████████| 50/50 [00:14<00:00,  3.35it/s]


adjusted Melody length: 43680
finding causality....
2328 true causes out of 3024 total connections
time elapsed:  246.67933081799856
iteration:  3
Starting surrogate generation...


...

adjusted Melody length: 43680
finding causality....
2327 true causes out of 3024 total connections
time elapsed:  240.7403798680025
iteration:  9
Starting surrogate generation...


100%|██████████| 50/50 [00:16<00:00,  3.08it/s]


adjusted Melody length: 43680
finding causality....
2405 true causes out of 3024 total connections
time elapsed:  235.35074236599758
iteration:  10
Starting surrogate generation...


100%|██████████| 50/50 [00:25<00:00,  1.96it/s]


adjusted Melody length: 43680
finding causality....
2141 true causes out of 3024 total connections
time elapsed:  245.82174390100408


In [22]:
avg_conns(tc29), avg_conns(t_dict["Surrogate29"]), avg_conns(t_dict["Causality29"])

(2322.1, 37.971970005199545, 197.75677085439892)

In [ ]:
t_dict

In [3]:
# gentimes8 = []
#
# for _ in range(10):
#     gentimes8.append(get_generation_time50('8', '8_d'))
#     gentimes8.append(get_generation_time50('15', '15_m'))
#     gentimes8.append(get_generation_time50('22', '22_a'))
#     gentimes8.append(get_generation_time50('28', '28_k'))
#     gentimes8.append(get_generation_time50('29', '29_h'))
#
#
# gentime8 = np.array(gentimes8[0:10]).mean()
# gentime15 = np.array(gentimes8[10:20]).mean()
# gentime22 = np.array(gentimes8[20:30]).mean()
# gentime28 = np.array(gentimes8[30:40]).mean()
# gentime29 = np.array(gentimes8[40:50]).mean()
#
# gentime8, gentime15, gentime22, gentime28, gentime29

Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting surrogate generation...
Starting s

## 150 surrogates

In [6]:
t_dict = {}

In [ ]:
tc15, totc15, ss15 = _ragaIterator(10, '15', '15_m', num_surrogates=150, times_it=t_dict)

iteration:  1
Starting surrogate generation...


100%|██████████| 150/150 [00:54<00:00,  2.74it/s]


finding causality....
20336 true causes out of 24024 total connections
time elapsed:  1912.7792934960016
iteration:  2
Starting surrogate generation...


100%|██████████| 150/150 [00:35<00:00,  4.24it/s]


finding causality....
19595 true causes out of 24024 total connections
time elapsed:  1888.3809031659985
iteration:  3
Starting surrogate generation...


...

finding causality....
20604 true causes out of 24024 total connections
time elapsed:  1900.1705254010012
iteration:  5
Starting surrogate generation...


100%|██████████| 150/150 [00:51<00:00,  2.91it/s]


finding causality....
20164 true causes out of 24024 total connections
time elapsed:  1918.1216722840036
iteration:  6
Starting surrogate generation...


100%|██████████| 150/150 [00:45<00:00,  3.26it/s]


finding causality....


In [ ]:
avg_conns(tc15), avg_conns(t_dict["Surrogate15"]), avg_conns(t_dict["Causality15"])

In [12]:
tc8, totc8, ss8 = _ragaIterator(10, '8', '8_d', num_surrogates=150, times_it=t_dict)

iteration:  1
Starting surrogate generation...
finding causality....
14463 true causes out of 24648 total connections
time elapsed:  2377.1097622410016
iteration:  2
Starting surrogate generation...
finding causality....
15204 true causes out of 24648 total connections
time elapsed:  2329.3596639659954
iteration:  3
Starting surrogate generation...
finding causality....
15244 true causes out of 24648 total connections
time elapsed:  2377.415402065002
iteration:  4
Starting surrogate generation...
finding causality....


Process ForkPoolWorker-1040:
Process ForkPoolWorker-1047:
Process ForkPoolWorker-1052:
Process ForkPoolWorker-1055:
Process ForkPoolWorker-1034:
Process ForkPoolWorker-1054:
Process ForkPoolWorker-1051:
Process ForkPoolWorker-1045:
Process ForkPoolWorker-1046:
Process ForkPoolWorker-1049:
Process ForkPoolWorker-1039:
Process ForkPoolWorker-1041:
Process ForkPoolWorker-1056:
Process ForkPoolWorker-1035:
Process ForkPoolWorker-1033:
Process ForkPoolWorker-1043:
Process ForkPoolWorker-1048:
Process ForkPoolWorker-1053:
Process ForkPoolWorker-1038:
Process ForkPoolWorker-1050:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/abhisheknandekar/miniconda3/envs/raga/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/abhisheknandekar/miniconda3

KeyboardInterrupt: 

In [ ]:
avg_conns(tc8), avg_conns(t_dict["Surrogate8"]), avg_conns(t_dict["Causality8"])

In [ ]:
tc22, totc22, ss22 = _ragaIterator(10, '22', '22_a', num_surrogates=150, times_it=t_dict)

In [ ]:
avg_conns(tc22), avg_conns(t_dict["Surrogate22"]), avg_conns(t_dict["Causality22"])

In [ ]:
tc28, totc28, ss28 = _ragaIterator(10, '28', '28_k', num_surrogates=150, times_it=t_dict)

In [ ]:
avg_conns(tc28), avg_conns(t_dict["Surrogate28"]), avg_conns(t_dict["Causality28"])

In [ ]:
tc29, totc29, ss29 = _ragaIterator(10, '29', '29_h', num_surrogates=150, times_it=t_dict)

In [ ]:
avg_conns(tc29), avg_conns(t_dict["Surrogate29"]), avg_conns(t_dict["Causality29"])

In [ ]:
t_dict

In [ ]:
t_dict = {}

In [ ]:
tc65, totc65, ss65 = _ragaIterator(10, '65', '29_h', num_surrogates=100, times_it=t_dict)

In [ ]:
avg_conns(tc65), avg_conns(t_dict["Surrogate65"]), avg_conns(t_dict["Causality65"])

In [ ]:
tc15, totc15, ss15 = _ragaIterator(10, '15', '15_m', num_surrogates=100, times_it=t_dict)

In [ ]:
avg_conns(tc15), avg_conns(t_dict["Surrogate15"]), avg_conns(t_dict["Causality15"])

In [ ]:
tc8, totc8, ss8 = _ragaIterator(10, '8', '8_d', num_surrogates=100, times_it=t_dict)

In [ ]:
avg_conns(tc8), avg_conns(t_dict["Surrogate8"]), avg_conns(t_dict["Causality8"])

In [ ]:
tc22, totc22, ss22 = _ragaIterator(10, '22', '22_a', num_surrogates=100, times_it=t_dict)

In [ ]:
avg_conns(tc22), avg_conns(t_dict["Surrogate22"]), avg_conns(t_dict["Causality22"])

In [ ]:
tc28, totc28, ss28 = _ragaIterator(10, '28', '28_k', num_surrogates=100, times_it=t_dict)

In [ ]:
avg_conns(tc28), avg_conns(t_dict["Surrogate28"]), avg_conns(t_dict["Causality28"])

In [ ]:
tc29, totc29, ss29 = _ragaIterator(10, '29', '29_h', num_surrogates=100, times_it=t_dict)

In [ ]:
avg_conns(tc29), avg_conns(t_dict["Surrogate29"]), avg_conns(t_dict["Causality29"])

In [ ]:
t_dict

